In [8]:
import psycopg2

# Replace these values with your PostgreSQL credentials and database name
db_host = "localhost"
db_user = "postgres"
db_password = "useruser"
db_name = "postgres"

# Establish a connection to the existing database
connection = psycopg2.connect(
    host=db_host,
    user=db_user,
    password=db_password,
    database=db_name
)

# Create a cursor object to execute SQL queries
cursor = connection.cursor()


# Create a table (if not exists)
create_table_query = '''
    CREATE TABLE IF NOT EXISTS example_table (
        id SERIAL PRIMARY KEY,
        name VARCHAR(255),
        age INT
    );
'''
cursor.execute(create_table_query)

# Insert data into the table
insert_data_query = "INSERT INTO example_table (name, age) VALUES (%s, %s);"
data_to_insert = [("Alice", 25), ("Bob", 30), ("Charlie", 22)]

for data in data_to_insert:
    cursor.execute(insert_data_query, data)

# Commit the changes
connection.commit()

# Select all data from the table
select_all_query = "SELECT * FROM example_table;"
cursor.execute(select_all_query)
rows = cursor.fetchall()

# Close the cursor and connection
cursor.close()
connection.close()

# Print the fetched rows
print(rows)


[(1, 'Alice', 25), (2, 'Bob', 30), (3, 'Charlie', 22)]


In [ ]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

# Assuming you have a DataFrame named 'df' and a PostgreSQL connection is established

# PostgreSQL connection parameters
db_params = {
    'host': 'localhost',
    'port': 5432,
    'user': 'postgres',
    'password': 'useruser',
    'database': 'postgres'
}

# Create a connection to the PostgreSQL database
connection = psycopg2.connect(**db_params)

# Create a SQLAlchemy engine
engine = create_engine(f'postgresql://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["database"]}')

# Replace 'entities_2325' with your actual table name
table_name = 'entities_2325'

# Transfer the DataFrame to the PostgreSQL database
df.to_sql(table_name, engine, if_exists='replace', index=False)

# Close the database connection
connection.close()

